In [1]:
import pandas as pd
import numpy as np
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import math
import h5py

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from shutil import copyfile
from copy import deepcopy

from code_base.utils import write_json, load_json

/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2024 Scored Data DF

In [4]:
train_metadata = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv1.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_prev_comp = pd.read_csv("/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV3_scored_meta_prev_comps_extended_2024SecLabels.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_xeno_canto = pd.read_csv("/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_noduplV3_extended_2024SecLabels.csv", converters={"secondary_labels":eval, "all_labels": eval})

In [ ]:
scored_birds = load_json("/home/vova/data/exps/birdclef_2024/scored_birds/sb_2024.json")

In [ ]:
def extract_sampler_col(row, scored_birds_):
    if row["primary_label"] in scored_birds_:
        return row["primary_label"]
    else:
        return list(set(row["secondary_labels"]) & set(scored_birds_))[0]

In [ ]:
train_metadata_prev_comp["stratify_col"] = train_metadata_prev_comp.apply(
    lambda row: extract_sampler_col(row, scored_birds),
    axis=1
)

In [ ]:
train_metadata["stratify_col"] = train_metadata["primary_label"]
train_metadata_xeno_canto["stratify_col"] = train_metadata_xeno_canto["primary_label"]

In [ ]:
train_metadata_prev_comp["stratify_col"].apply(lambda x: x in scored_birds).all()

In [ ]:
# train_metadata_prev_comp.to_csv(
#     "/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV2_scored_meta_prev_comps_extended.csv",
#     index=False
# )

In [ ]:
# train_metadata.to_csv(
#     "/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended.csv",
#     index=False
# )

In [ ]:
# train_metadata_xeno_canto.to_csv(
#     "/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_nodupl_extended.csv",
#     index=False
# )

In [ ]:
all_labels = pd.concat([
    train_metadata["primary_label"], train_metadata_prev_comp["stratify_col"], train_metadata_xeno_canto["primary_label"]
]).reset_index(drop=True)

## "Power" Balanced

In [ ]:
sample_weights = (all_labels.value_counts() / all_labels.value_counts().sum())  ** (-0.5)

In [ ]:
sample_weights

In [ ]:
write_json("/home/vova/data/exps/birdclef_2024/sample_weights/sw_2024_add_data_v1.json", sample_weights.to_dict())

# Simple Balanced

In [6]:
all_labels = pd.concat([
    train_metadata["stratify_col"], train_metadata_prev_comp["stratify_col"], train_metadata_xeno_canto["stratify_col"]
]).reset_index(drop=True)

In [13]:
sample_weights = (all_labels.value_counts() / all_labels.value_counts().sum()) ** (-1)

In [14]:
all_labels.value_counts()

stratify_col
houspa     2618
eaywag1    2474
commoo3    2256
eurcoo     2157
barswa     1753
           ... 
integr       11
blaeag1      11
wbbfly1       9
niwpig1       8
asiope1       7
Name: count, Length: 182, dtype: int64

In [15]:
sample_weights

stratify_col
houspa       18.695951
eaywag1      19.784155
commoo3      21.695922
eurcoo       22.691701
barswa       27.921278
              ...     
integr     4449.636364
blaeag1    4449.636364
wbbfly1    5438.444444
niwpig1    6118.250000
asiope1    6992.285714
Name: count, Length: 182, dtype: float64

In [16]:
write_json("/home/vova/data/exps/birdclef_2024/sample_weights/sw_2024_add_data_v2_bal.json", sample_weights.to_dict())

# 2024 + 2024 Scored from XC + Prev Comps

In [17]:
train_metadata = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv1.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_prev_comp = pd.read_csv("/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV2_meta_prev_comps_extended_pruned.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_xeno_canto = pd.read_csv("/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_noduplV3_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})

/tmp/ipykernel_1175714/1563590999.py:2: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata_prev_comp = pd.read_csv("/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV2_meta_prev_comps_extended_pruned.csv", converters={"secondary_labels":eval, "all_labels": eval})


In [18]:
all_labels = pd.concat([
    train_metadata["primary_label"], train_metadata_prev_comp["primary_label"], train_metadata_xeno_canto["primary_label"]
]).reset_index(drop=True)

In [19]:
sample_weights = (all_labels.value_counts() / all_labels.value_counts().sum())  ** (-0.5)

In [20]:
write_json("/home/vova/data/exps/birdclef_2024/sample_weights/sw_2024_PrevComp_v1.json", sample_weights.to_dict())

In [21]:
sample_weights

primary_label
houspa       7.334228
eaywag1      7.546349
commoo3      7.897756
eurcoo       8.076965
redcro       8.488328
              ...    
yebsto1    375.122647
maupar     375.122647
brtcha1    375.122647
crefra2    375.122647
lotcor1    375.122647
Name: count, Length: 992, dtype: float64

In [23]:
all_labels.isna().sum()

0